| SOURCE_TABLE_NAME | SOURCE_COLUMN_NAME | REPORTING_TABLE_NAME | REPORTING_COLUMN_NAME | TRANSFORMATION RULE | CONDITIONS |
| --- | --- |--- | --- |--- |--- |
| silver.daily_pricing_silver	| state_name	|***REPORTING_DIM_STATE_GOLD***	|***STATE_NAME***	| Select Unique state_name Values | Identify New/Changed Records From the Source Table . Use ***lakehouse_updated_date Column*** in source table to Identify New/Changed Records|
| DERIVED	| DERIVED	|***REPORTING_DIM_STATE_GOLD***	| ***STATE_ID***	| Generate Running Sequnece ID For Each Unique state_name Values | 1. Make Sure No Duplicate State_Name Values Loaded 2.  When loading new State_Name values in subsquent run , STATE_ID values need to be generated on top of existing MAX(STATE_ID)   |
| DERIVED	| DERIVED	|***REPORTING_DIM_STATE_GOLD***	| ***lakehouse_inserted_date***	| Load current_timestamp() | |
| DERIVED	| DERIVED	|***REPORTING_DIM_STATE_GOLD***	| ***lakehouse_updated_date***	| Load current_timestamp() | |

In [0]:
create or replace table `pricing-analytics`.silver.reporting_dim_state_stage_1
as
select distinct state_name from `pricing-analytics`.silver.daily_pricing_silver
where LAKEHOUSE_UPDATED_DATE > (select nvl(max(processed_table_datetime),'1900-01-01') from `pricing-analytics`.processrunlogs.deltalakehouse_process_runs where process_name='reportingDimensionsTablesLoad' and process_status='Completed')

In [0]:
create or replace table `pricing-analytics`.silver.reporting_dim_state_stage_2
as
select rdss1.state_name
, row_number() over (order by rdss1.state_name) + msi.max_state_id as state_id
, current_timestamp() as lakehouse_inserted_date
, current_timestamp() as lakehouse_updated_date
from `pricing-analytics`.silver.reporting_dim_state_stage_1 rdss1
left join `pricing-analytics`.gold.reporting_dim_state_gold rdsg
on rdss1.state_name=rdsg.state_name
cross join (select nvl(max(state_id),0) as max_state_id from `pricing-analytics`.gold.reporting_dim_state_gold) msi
where rdsg.state_name is null
;

In [0]:
insert into `pricing-analytics`.gold.reporting_dim_state_gold
select * from `pricing-analytics`.silver.reporting_dim_state_stage_2;

In [0]:
CREATE OR REPLACE TABLE `pricing-analytics`.silver.reporting_dim_market_stage_1 AS
SELECT 
 DISTINCT MARKET_NAME
FROM `pricing-analytics`.silver.daily_pricing_silver
WHERE lakehouse_updated_date > (SELECT nvl(max(PROCESSED_TABLE_DATETIME),'1900-01-01') FROM `pricing-analytics`.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE process_name = 'reportingDimensionTablesLoad' AND process_status = 'Completed' )

In [0]:
CREATE OR REPLACE TABLE `pricing-analytics`.silver.reporting_dim_market_stage_2 AS 
SELECT 
  silverDim.MARKET_NAME
 ,ROW_NUMBER() OVER (  ORDER BY silverDim.MARKET_NAME)  as MARKET_ID
 ,current_timestamp() as lakehouse_inserted_date
 ,current_timestamp() as lakehouse_updated_date
FROM `pricing-analytics`.silver.reporting_dim_market_stage_1 silverDim
LEFT OUTER JOIN `pricing-analytics`.gold.reporting_dim_market_gold goldDim
ON silverDim.MARKET_NAME = goldDim.MARKET_NAME
WHERE goldDim.MARKET_NAME IS NULL;

In [0]:
CREATE OR REPLACE TABLE `pricing-analytics`.silver.reporting_dim_market_stage_3 AS 
SELECT
silverDim.MARKET_NAME 
,silverDim.MARKET_ID + PREV_MAX_SK_ID as MARKET_ID
,current_timestamp() as lakehouse_inserted_date
,current_timestamp() as lakehouse_updated_date
FROM 
`pricing-analytics`.silver.reporting_dim_market_stage_2 silverDim
CROSS JOIN (SELECT NVL(MAX(MARKET_ID),0) as PREV_MAX_SK_ID FROM `pricing-analytics`.gold.reporting_dim_market_gold ) goldDim;

In [0]:
INSERT INTO `pricing-analytics`.gold.reporting_dim_market_gold
SELECT
MARKET_NAME
,MARKET_ID
,current_timestamp() 
,current_timestamp() 
FROM `pricing-analytics`.silver.reporting_dim_market_stage_3;

In [0]:
CREATE OR REPLACE TABLE `pricing-analytics`.silver.reporting_dim_variety_stage_1 AS
SELECT 
 DISTINCT VARIETY
FROM `pricing-analytics`.silver.daily_pricing_silver
WHERE lakehouse_updated_date > (SELECT nvl(max(PROCESSED_TABLE_DATETIME),'1900-01-01') FROM `pricing-analytics`.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE process_name = 'reportingDimensionTablesLoad' AND process_status = 'Completed' );

In [0]:
CREATE OR REPLACE TABLE `pricing-analytics`.silver.reporting_dim_variety_stage_2 AS 
SELECT 
  silverDim.VARIETY
 ,ROW_NUMBER() OVER (  ORDER BY silverDim.VARIETY)  as VARIETY_ID
 ,current_timestamp() as lakehouse_inserted_date
 ,current_timestamp() as lakehouse_updated_date
FROM `pricing-analytics`.silver.reporting_dim_variety_stage_1 silverDim
LEFT OUTER JOIN `pricing-analytics`.gold.reporting_dim_variety_gold goldDim
ON silverDim.VARIETY= goldDim.VARIETY
WHERE goldDim.VARIETY IS NULL;

In [0]:
CREATE OR REPLACE TABLE `pricing-analytics`.silver.reporting_dim_variety_stage_3 AS 
SELECT
silverDim.VARIETY 
,silverDim.VARIETY_ID + PREV_MAX_SK_ID as VARIETY_ID
,PREV_MAX_SK_ID
,current_timestamp() as lakehouse_inserted_date
,current_timestamp() as lakehouse_updated_date
FROM 
`pricing-analytics`.silver.reporting_dim_variety_stage_2 silverDim
CROSS JOIN (SELECT nvl(MAX(VARIETY_ID),0) as PREV_MAX_SK_ID FROM `pricing-analytics`.gold.reporting_dim_variety_gold ) goldDim;

In [0]:
INSERT INTO `pricing-analytics`.gold.reporting_dim_variety_gold
SELECT
VARIETY
,VARIETY_ID
,current_timestamp() 
,current_timestamp() 
FROM `pricing-analytics`.silver.reporting_dim_variety_stage_3

In [0]:
CREATE OR REPLACE TABLE `pricing-analytics`.silver.reporting_dim_product_stage_1 AS
SELECT 
 DISTINCT PRODUCT_NAME
 ,PRODUCTGROUP_NAME
FROM `pricing-analytics`.silver.daily_pricing_silver
WHERE lakehouse_updated_date > (SELECT nvl(max(PROCESSED_TABLE_DATETIME),'1900-01-01') FROM `pricing-analytics`.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE process_name = 'reportingDimensionTablesLoad' AND process_status = 'Completed' );

In [0]:
CREATE OR REPLACE TABLE `pricing-analytics`.silver.reporting_dim_product_stage_2 AS 
SELECT 
  silverDim.PRODUCT_NAME
  ,silverDim.PRODUCTGROUP_NAME
 ,ROW_NUMBER() OVER (  ORDER BY silverDim.PRODUCT_NAME,silverDim.PRODUCTGROUP_NAME)  as PRODUCT_ID
 ,current_timestamp() as lakehouse_inserted_date
 ,current_timestamp() as lakehouse_updated_date
FROM `pricing-analytics`.silver.reporting_dim_product_stage_1 silverDim
LEFT OUTER JOIN `pricing-analytics`.gold.reporting_dim_product_gold goldDim
ON silverDim.PRODUCT_NAME= goldDim.PRODUCT_NAME
AND silverDim.PRODUCTGROUP_NAME = goldDim.PRODUCTGROUP_NAME
WHERE goldDim.PRODUCT_NAME IS NULL;

In [0]:
CREATE OR REPLACE TABLE `pricing-analytics`.silver.reporting_dim_product_stage_3 AS 
SELECT
  silverDim.PRODUCT_NAME
  ,silverDim.PRODUCTGROUP_NAME
,silverDim.PRODUCT_ID + PREV_MAX_SK_ID as PRODUCT_ID
,PREV_MAX_SK_ID
,current_timestamp() as lakehouse_inserted_date
,current_timestamp() as lakehouse_updated_date
FROM 
`pricing-analytics`.silver.reporting_dim_product_stage_2 silverDim
CROSS JOIN (SELECT nvl(MAX(PRODUCT_ID),0) as PREV_MAX_SK_ID FROM `pricing-analytics`.gold.reporting_dim_product_gold ) goldDim;

In [0]:
INSERT INTO `pricing-analytics`.gold.reporting_dim_product_gold
SELECT
 PRODUCTGROUP_NAME
,PRODUCT_NAME
,PRODUCT_ID
,current_timestamp() 
,current_timestamp() 
FROM `pricing-analytics`.silver.reporting_dim_product_stage_3

In [0]:
INSERT INTO  `pricing-analytics`.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(PROCESS_NAME,PROCESSED_TABLE_DATETIME,PROCESS_STATUS)
SELECT 'reportingDimensionTablesLoad' , max(lakehouse_updated_date) ,'Completed' FROM `pricing-analytics`.silver.daily_pricing_silver

In [0]:
select * from `pricing-analytics`.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS